In [ ]:
# import necessary library
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [ ]:
user_login_history_table_columns = [
    'id',
    'is_mobile',
    'client_ip',
    'device_name',
    'browser_name',
    'employee_record_id',
    'ministry_id',
    'layer_id',
    'origin_id',
    'office_id',
    'office_unit_id',
    'office_unit_organogram',
    'office_name',
    'unit_name',
    'organogram_name',
    'user_id',
    'employee_name',
    'mobile_number',
    'employee_email',
    'login_time',
    'logout_time',
    'network_information',
    'created',
    'modified',
    'created_by',
    'modified_by',
    'token',
    'device_type',
    'device_id',
]
len(user_login_history_table_columns)

In [ ]:
# SELECT count(id) FROM `user_login_history` where date(created) >= '2020-09-01'  and date(created) <= '2020-09-30' and is_mobile = 1;

In [ ]:

user_login_history_df = pd.read_csv('../Data/user_login_history_24_11_2021.csv', names=user_login_history_table_columns)
user_login_history_df.shape

In [ ]:
user_login_history_df.columns

In [ ]:
user_login_history_df.info(memory_usage='deep')

In [ ]:
x = user_login_history_df.memory_usage(deep=True)/1024/1024
x.sort_values(ascending=False)

In [ ]:
# type(x)
# x.values

# Most logged in users

In [ ]:
# user_login_history_df[['user_id', 'employee_name']]
# x = user_login_history_df.loc[user_login_history_df.user_id == 97804]
# x.shape
# # x.tail()
# set(x.employee_name.values)
# group_by_user_id = user_login_history_df.groupby('user_id')
# # for group_name, group_frame in group_by_user_id:
# #     group_name, group_frame.size
# group_by_user_id.size().sort_values(ascending=False)[:10]

# Most logged in users

### Android Users vs IOS Users

In [ ]:
user_login_history_df.shape
x = user_login_history_df.loc[(user_login_history_df.device_type == 'android') | (user_login_history_df.device_type == 'IOS')]
x.device_type.unique()

x['device_type'].value_counts().plot(kind='bar')


### Android Users vs IOS Users

In [ ]:
user_login_history_df.head()

### Make mobile Users dataframe with limited attribute

In [ ]:
# mobile_users_df = user_login_history_df[['id', 'is_mobile', 'employee_record_id', 'created', 'modified']]
# mobile_users_df = mobile_users_df.copy(deep=True)
# mobile_users_df = mobile_users_df[mobile_users_df.is_mobile == 1].copy(deep=True)
# user_login_history_df.device_type.unique()
user_login_history_df[user_login_history_df.device_type == 'android'].shape
user_login_history_df[user_login_history_df.device_type == 'IOS']

In [ ]:
mobile_users_df

In [ ]:
x = mobile_users_df[mobile_users_df.created.isnull()]
type(x)
x.shape

In [ ]:
# dir(x)
# len(x.to_list())

In [ ]:
mobile_users_df["created"].fillna("unknown", inplace = True)


In [ ]:
import re
year_month_regex = re.compile(r'(\d\d\d\d)-(\d\d)-(\d\d)')

In [ ]:
# users_df.info()
years = []
months = []
indexes = []

times = mobile_users_df.created.values
for index, time in enumerate(times, start=0):
    if time == 'unknown':
        year = time
        month = time
        years.append(year)
        months.append(month)
        indexes.append(index)
    else:
        # print(time)
        mo = year_month_regex.search(time)
        year = mo.group(1)
        month = mo.group(2)
        years.append(year)
        months.append(month)
        indexes.append(index)

In [ ]:
years[:5]
months[:5]

In [ ]:
year_series_df = pd.Series(years, index=indexes)
month_series_df = pd.Series(months, index=indexes)
# year_series_df[:10]
# month_series_df[:10]

In [ ]:
# add year and month dataframe columns
mobile_users_df['year'] = year_series_df
mobile_users_df['month'] = month_series_df

In [ ]:
# save users_df as users_updated.csv
mobile_users_df.to_csv('mobile_users_df.csv', index=False)

In [ ]:
month_map ={
    '1': 'January',
    '01': 'January',
    '2': 'February',
    '02': 'February',
    '3': 'March',
    '03': 'March',
    '4': 'April',
    '04': 'April',
    '5': 'May',
    '05': 'May',
    '6': 'June',
    '06': 'June',
    '7': 'July',
    '07': 'July',
    '8': 'August',
    '08': 'August',
    '9': 'Septembr',
    '09': 'Septermber',
    '10': 'October',
    '11': 'November',
    '12': 'December',
    'unknown': 'unknown',
}

In [ ]:
mobile_users_df_year_by = mobile_users_df.groupby('year')
mobile_users_df_year_by

In [ ]:
general_series = []
drilldown_series = []
for year, year_frame in mobile_users_df_year_by:
    # group, frame.shape[0]
    year = str(year)
    
    temporary_general_dict = {
        'name': year,
        'y': year_frame.shape[0],
        'drilldown': year
    }
    general_series.append(temporary_general_dict)
    temporary_drilldown_dict = {
        'name': year,
        'id': year,
        'data': [],
    }
    month_group_by = year_frame.groupby('month')
    for month, month_frame in month_group_by:
        # mg, mf.shape[0]
        month = str(month)
        month = month_map[month]
        # print(month)
       
        # month, month_frame.shape[0]
        lst = [month, month_frame.shape[0]]
        temporary_drilldown_dict['data'].append(lst)
        drilldown_series.append(temporary_drilldown_dict)
    
    # print()
    # print()

In [ ]:
general_series
drilldown_series

In [ ]:
generic_user_login_history_df = user_login_history_df[[
    'id',
    'is_mobile',
    'device_name',
    'origin_id',
    'ofice_id',
    'office_unit_id',
    'office_unit_organogram',
    'user_id',
    'created',
    'modified',
]]

In [ ]:
generic_user_login_history_df

In [ ]:
# user_login_history_df.is_mobile.unique()
# user_login_history_df.is_mobile
generic_user_login_history_df[generic_user_login_history_df.created.isnull()]
generic_user_login_history_df[generic_user_login_history_df.created.isnull()].shape
